In [104]:
import os

import numpy as np
import pandas as pd
import pickle 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Plots
import matplotlib.pyplot as plt

# Prerocessing for FEDOT
from fedot.core.data.data import InputData
from fedot.core.repository.dataset_types import DataTypesEnum
from fedot.api.main import Fedot
from fedot.core.repository.tasks import Task, TaskTypesEnum, TsForecastingParams

# FEDOT 
from fedot.core.pipelines.pipeline import Pipeline
from fedot.core.pipelines.node import PrimaryNode, SecondaryNode

import logging
logging.raiseExceptions = False

Again load data and prepare as for manual stage (see data_analysis.ipynb)

In [128]:
df = None
with open("covid_flow.pkl", "rb") as f:
    df = pickle.load(f)
df["case"] = df.index
df["t_point"] = df["t_point"].apply(lambda x: x[2:])
df["t_point"] = df["t_point"].apply(lambda x: int(x))
df = df.set_index(["case", "t_point"])
df = df.sort_values(["case", "t_point"])

Lets take window size=3 and now we will use SHIFT to make more train data

In [106]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
dinam_fact_df = df.iloc[:,29:42]
dinam_fact_df = dinam_fact_df.sort_values(["case", "t_point"])
imp = IterativeImputer(estimator=LinearRegression(),
                       missing_values=np.nan, max_iter=50, imputation_order='roman')
df_imputed = imp.fit_transform(dinam_fact_df)
dinam_fact_df = pd.DataFrame(data=df_imputed, columns = dinam_fact_df.columns, index=dinam_fact_df.index)
dinam_fact_df

Температура_dinam_fact  Лимфоциты#_dinam_fact  \
case    t_point                                                  
GACAAcY 0                     36.700000                   1.93   
        1                     36.300000                   2.00   
        2                     36.600000                   2.13   
        3                     36.500000                   2.12   
GACAAnf 0                     36.701477                   1.21   
...                                 ...                    ...   
GACBB;O 0                     36.900000                   1.22   
        1                     36.500000                   1.65   
GACXAAB 0                     36.500000                   1.11   
        1                     36.900000                   1.60   
        2                     36.600000                   3.45   

                 АСТ_dinam_fact  ЧСС_dinam_fact  ЧДД_dinam_fact  \
case    t_point                                                   
GACAAcY 0                 27.90       74.000000       24.000000   
        1                 25.70       71.000000       19.000000   
        2                 32.00       71.699426       18.000000   
        3                 27.00       71.000000       18.000000   
GACAAnf 0                 20.70       55.000000       22.000000   
...                         ...             ...             ...   
GACBB;O 0                 22.34       54.000000       18.000000   
        1                244.03       69.291345       17.000000   
GACXAAB 0                 21.80       69.482981       19.773952   
        1                 60.80       75.000000       16.000000   
        2                 17.20       73.013739       15.000000   

                 Билирубин общий_dinam_fact  \
case    t_point                               
GACAAcY 0                         15.300000   
        1                         12.100000   
        2                         10.480305   
        3                         11.298666   
GACAAnf 0                          7.800000   
...                                     ...   
GACBB;O 0                          1.430000   
        1                          3.780000   
GACXAAB 0                          4.500000   
        1                          1.100000   
        2                          9.300000   

                 MPV- Средн.объем тромбоцитов_dinam_fact  \
case    t_point                                            
GACAAcY 0                                           11.2   
        1                                           10.6   
        2                                           11.0   
        3                                           11.4   
GACAAnf 0                                            8.1   
...                                                  ...   
GACBB;O 0                                            7.0   
        1                                            7.2   
GACXAAB 0                                           10.6   
        1                                           10.8   
        2                                           10.0   

                 PCT- Тромбокрит_dinam_fact  Лимфоциты%_dinam_fact  \
case    t_point                                                      
GACAAcY 0                             0.100                   48.3   
        1                             0.160                   30.7   
        2                             0.190                   23.1   
        3                             0.100                   24.1   
GACAAnf 0                             0.201                   24.1   
...                                     ...                    ...   
GACBB;O 0                             0.134                   33.0   
        1                             0.210                   41.2   
GACXAAB 0                             0.200                   18.8   
        1                             0.270                   12.9   
        2                             0.280          

In [107]:
def split_features_targets(df, window=3):
    X, y = [], []
    for i in df.index.levels[0]:
        len_ts = dinam_fact_df.groupby(["case"]).size()[i]
        if len_ts < window+1:
            continue
        X.append(df.loc[(i, 0): (i, window-1)].to_numpy().reshape(window*13))
        y.append(df.loc[(i, window)].to_numpy())
    return np.array(X), np.array(y)

X, y = split_features_targets(dinam_fact_df)
X, X_test, y, y_test = train_test_split(X, y, test_size=0.1)

In [113]:
y[0]

array([3.66252537e+01, 2.98781184e-01, 1.62979056e+02, 7.90000000e+01,
       2.51464482e+01, 1.23244179e+01, 8.45569417e+00, 1.04035436e-01,
       3.51766169e+01, 2.00000000e+00, 4.00000000e+00, 5.21595269e+02,
       1.84053482e+01])

In [108]:
model = Fedot(problem='regression', timeout=30, n_jobs=-1)
obtained_pipeline = model.fit(features=X, target=y)
obtained_pipeline.show()


2022-11-06 23:11:50,175 - ApiComposer - Initial pipeline was fitted in 1.7 sec.
2022-11-06 23:11:50,182 - ApiComposer - AutoML configured. Parameters tuning: True Time limit: 30 min Set of candidate models: ['adareg', 'dtreg', 'gbr', 'knnreg', 'lasso', 'lgbmreg', 'linear', 'rfr', 'ridge', 'sgdr', 'svr', 'treg', 'scaling', 'normalization', 'pca', 'fast_ica', 'poly_features', 'ransac_lin_reg', 'ransac_non_lin_reg', 'isolation_forest_reg']
2022-11-06 23:11:50,216 - ApiComposer - Pipeline composition started.


Generations:   0%|          | 1/10000 [00:00<?, ?gen/s]

2022-11-07 10:19:01,176 - GroupedCondition - Optimisation stopped: Time limit is reached


Generations:   0%|          | 1/10000 [11:07:10<?, ?gen/s]

2022-11-07 10:19:01,188 - ApiComposer - Time for pipeline composing was 11:07:10.972407.
The remaining 637.2106656 seconds are not enough to tune the hyperparameters.
2022-11-07 10:19:01,190 - ApiComposer - Composed pipeline returned without tuning.


2022-11-07 10:19:01,729 - ApiComposer - Model generation finished


C:\Users\sirau\OneDrive\Desktop\datascience\covid\venv\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
C:\Users\sirau\OneDrive\Desktop\datascience\covid\venv\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ValueError: Found array with 1 sample(s) (shape=(1, 212)) while a minimum of 2 is required by FastICA.

In [ ]:
print("Scaling + RandomForestRegression results:")
y_pred = model.predict(X_test)
r2_res = r2_score(y_test, y_pred, multioutput="raw_values")
mse = mean_squared_error(y_test, y_pred)
display(pd.DataFrame(data=r2_res.reshape(1, 13), columns=dinam_fact_df.columns, index=["R2"]))
print("RMSE ", mse**(1/2))

Now we will try to add some statical facts. Age, Medicines, Treatments

In [140]:
df.loc[("GACAk+Q", 0)].iloc[43:47].to_numpy().reshape(4)

array([0, 0, 0, 0], dtype=object)

In [141]:
def split_features_targets(df, dinam_fact_df, window=3):
    X, y = [], []
    for i in df.index.levels[0]:
        len_ts = dinam_fact_df.groupby(["case"]).size()[i]
        if len_ts < window+1:
            continue
        sample_data = dinam_fact_df.loc[(i, 0): (i, window-1)].to_numpy().reshape(window*13) # facts
        sample_data = np.append(sample_data, df.loc[(i, 0): (i, window-1)].iloc[:, 43:47].to_numpy().reshape(window*4)) #controls
        sample_data = np.append(sample_data, df.loc[(i, 0)].iloc[:29].to_numpy()) #medicines
        sample_data = np.append(sample_data, df.loc[(i, 0)].iloc[42]) #age
        X.append(sample_data)
        y.append(dinam_fact_df.loc[(i, window)].to_numpy())
    return np.array(X), np.array(y)

X, y = split_features_targets(df, dinam_fact_df)
X, X_test, y, y_test = train_test_split(X, y, test_size=0.1)

In [152]:
y.shape

(477, 13)

In [ ]:
model_2 = Fedot(problem='regression', timeout=5, n_jobs=-1)
obtained_pipeline = model.fit(features=X, target=y)
obtained_pipeline.show()

2022-11-07 11:26:45,027 - ApiComposer - Initial pipeline was fitted in 1.5 sec.
2022-11-07 11:26:45,032 - ApiComposer - AutoML configured. Parameters tuning: True Time limit: 30 min Set of candidate models: ['adareg', 'dtreg', 'gbr', 'knnreg', 'lasso', 'lgbmreg', 'linear', 'rfr', 'ridge', 'sgdr', 'svr', 'treg', 'scaling', 'normalization', 'pca', 'fast_ica', 'poly_features', 'ransac_lin_reg', 'ransac_non_lin_reg', 'isolation_forest_reg']
2022-11-07 11:26:45,054 - ApiComposer - Pipeline composition started.


Generations:   0%|          | 1/10000 [00:00<?, ?gen/s]

In [ ]:
print("Add static variables:")
y_pred = model_2.predict(X_test)
r2_res = r2_score(y_test, y_pred, multioutput="raw_values")
mse = mean_squared_error(y_test, y_pred)
display(pd.DataFrame(data=r2_res.reshape(1, 13), columns=dinam_fact_df.columns, index=["R2"]))
print("RMSE ", mse**(1/2))